In [1]:
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.stem import PorterStemmer

In [4]:
df = pd.read_csv('\\Users\Dell\Desktop\KCC-Cotton-Attack-QnA-Processes-Cleaned_v1.csv')

In [14]:

query21 = df.Ouery_Cleaned.apply(lambda x: '    ~[{0}]'.format(x))

In [15]:
query21.to_csv('KCC-Cotton-Attack-Query.csv')

C:\Users\Dell\Anaconda3\envs\data-x\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [3]:
df=pd.read_csv('KCC-Maharashtra-2Years.csv')

In [4]:
df_cotton = df[df.Crop == 'Cotton (Kapas)']

In [5]:
df_cotton.head()

,Season,Sector,Category,Crop,QueryType,Query,Answer,State,District,Block,DateTime
667,NaN,AGRICULTURE,Fiber Crops,Cotton (Kapas),Seeds,Varities of COTTON crop?,"Cotton- Chaitanya, bindast,rashi-659, kaveri,...",MAHARASHTRA,RATNAGIRI,DAPOLI,2016-04-10 17:14:20.543
1062,NaN,AGRICULTURE,Fiber Crops,Cotton (Kapas),\tPlant Protection\t,thrips,Spray confidor 10ml/15 lit of water,MAHARASHTRA,NASIK,IGATPURI,2015-04-02 09:46:10.847
1090,RABI,AGRICULTURE,Fiber Crops,Cotton (Kapas),\tField Preparation\t,Asked about Weather report ?,cloudy weather no chance of rainfall,MAHARASHTRA,NASIK,NASHIK,2015-04-02 10:27:25.350
1441,JAYAD,AGRICULTURE,Fiber Crops,Cotton (Kapas),Varieties,ASKED ABOUT VARIETIES OF COTTON CROP,"AJIT 155, AJIT 177, KANAK, RASI-2, BRAMHA VARI...",MAHARASHTRA,NASIK,BAGLAN,2015-04-04 13:03:49.903
1896,RABI,AGRICULTURE,Fiber Crops,Cotton (Kapas),\tPlant Protection\t,attack pest,????? – ??????? ??????? ???? ?????? ?? ???? ??...,MAHARASHTRA,NASIK,SINNAR,2015-04-06 17:09:55.490


In [6]:
df_protect = df_cotton[df_cotton.QueryType.str.contains('Protection')]

In [7]:
ps = PorterStemmer()

In [109]:
symptom_token =[]
sym_stopwords = set(stopwords.words("english"))
for symptom in df_protect.Query[:1000]:
    symptom = re.sub("[^a-zA-Z]", " ",symptom).lower().split()
    clean_symptom=[]
    for word in symptom:
        if (word not in sym_stopwords) & (word not in ['asked','cotton']):
            #word=ps.stem(word)
            clean_symptom.append(word)
    symptom_processed = ' '.join(clean_symptom)
    symptom_token.append(symptom_processed) 

In [110]:
vec = CountVectorizer(stop_words=set(stopwords.words("english")))

In [111]:
vec.fit_transform(symptom_token)

<1000x182 sparse matrix of type '<class 'numpy.int64'>'
	with 2233 stored elements in Compressed Sparse Row format>

RASA_NLU for KCC dataset

In [16]:
from rasa_nlu.training_data import load_data
from rasa_nlu import config
from rasa_nlu.model import Trainer
import json

In [17]:
training_data = load_data('training_dataset.json')

In [18]:
import spacy

In [19]:
trainer = Trainer(config.load("nlu-config.yml"))


C:\Users\Dell\Anaconda3\envs\data-x\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Dell\Anaconda3\envs\data-x\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Dell\Anaconda3\envs\data-x\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Dell\Anaconda3\envs\data-x\lib\site-packages\tensorflow\python\f

In [20]:
interpreter = trainer.train(training_data)




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dropout instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Epochs: 100%|█████████████████████████████████████████████████| 300/300 [00:13<00:00, 22.06it/s, loss=0.010, acc=1.000]


In [23]:
parsed = interpreter.parse('attakk of termtes?')

In [24]:
parsed

{'intent': {'name': 'plantprotectionsolution',
  'confidence': 0.7874523401260376},
 'entities': [{'start': 10,
   'end': 17,
   'value': 'termtes',
   'entity': 'disease',
   'confidence': 0.7846654337507237,
   'extractor': 'CRFEntityExtractor'}],
 'intent_ranking': [{'name': 'plantprotectionsolution',
   'confidence': 0.7874523401260376},
  {'name': 'bye', 'confidence': 0.39991292357444763},
  {'name': 'negative', 'confidence': 0.03889641910791397},
  {'name': 'greet', 'confidence': 0.0},
  {'name': 'affirmative', 'confidence': 0.0}],
 'text': 'attakk of termtes?'}

In [1]:
import tensorflow as tf
tf.__version__

C:\Users\Dell\Anaconda3\envs\data-x\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Dell\Anaconda3\envs\data-x\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Dell\Anaconda3\envs\data-x\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Dell\Anaconda3\envs\data-x\lib\site-packages\tensorflow\python\f

'1.14.0'

In [33]:
import pickle

with open('intent_classifier_sklearn.pkl', 'rb') as f:
    data = pickle.load(f)

C:\Users\Dell\Anaconda3\envs\data-x\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.19.1 when using version 0.20.4. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Dell\Anaconda3\envs\data-x\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator SVC from version 0.19.1 when using version 0.20.4. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Dell\Anaconda3\envs\data-x\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator GridSearchCV from version 0.19.1 when using version 0.20.4. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [35]:
data = pd.read_pickle('intent_classifier_sklearn.pkl')

In [37]:
data.()

TypeError: load() missing 1 required positional argument: 'meta'